In [0]:
from bs4 import BeautifulSoup
from urllib2 import urlopen
import urllib2
import pandas as pd
import numpy as np
import csv
import requests
from requests.exceptions import HTTPError

import sys
reload(sys)
sys.setdefaultencoding('utf-8')
 
lyricsvector = [] #input (bag of words)
genrevector = [] #target
songinfovector = []  #metadata (artist and songname)

# List the URLs here
urllist = [
"http://www.songlyrics.com/news/top-genres/christian/",
"http://www.songlyrics.com/news/top-genres/country-music/",
"http://www.songlyrics.com/news/top-genres/hip-hop-rap/",
"http://www.songlyrics.com/news/top-genres/rhythm-blues/",
"http://www.songlyrics.com/news/top-genres/pop/",
"http://www.songlyrics.com/news/top-genres/rock/"]


In [0]:

for i in range(0,6):
    req = urllib2.Request(urllist[i], headers={'User-Agent' : "Magic Browser"}) 
    url = urllib2.urlopen(req)
    doc = url.read()
    soup = BeautifulSoup(doc, 'html.parser')
    div = soup.find( 'div', { 'class': 'box listbox' } )
    #print(div)
 
# get genres
    title = soup.title.get_text().encode('ascii', 'ignore').decode().split(' ')
    index100 = title.index('100')
    indexSongs = title.index('Songs')
    genre = ' '.join(title[(index100+1):(indexSongs)]).encode('utf-8')
 
# create list of top 100 songs by genre
    print(genre)
    songs = div.find_all('a')
    songlinks = []

# create loop to extract song links
    for j in range(0,200): #[0::2]:
        songlink = songs[j].get('href').encode('ascii', 'ignore')
        songlinks.append(songlink) #output links to a list called songlinks

    songlinks = filter(None, songlinks)
    songlinks = [songlink for songlink in songlinks if (len(songlink.decode().split('/'))==6)]
    
    count=0
    for k in range(0,len(songlinks)):
        try:
            r = requests.get(songlinks[k].decode())
            r.raise_for_status()
        except HTTPError:
            print(songlinks[k].decode()+"- Error while retreiving page")
        else:
            req = urllib2.Request(songlinks[k].decode(), headers={'User-Agent' : "Magic Browser"}) 
            songurl = urllib2.urlopen(req)
            songdoc = songurl.read()
            songsoup = BeautifulSoup(songdoc, 'html.parser')
            songinfo = songsoup.title.get_text().encode('ascii', 'ignore')
            print(songinfo, 'is number', count)
            count=count+1

            songdiv = songsoup.find( 'div', { 'id': 'songLyricsDiv-outer' } )
            lyrics = songdiv.getText().replace("\n", " ").replace("\'", "").replace("\r", " ")

            lyricsvector.append(lyrics.decode())
            genrevector.append(genre.decode())
            songinfovector.append(songinfo.decode())
    

In [0]:
df = pd.DataFrame({ "Genre" : genrevector, "Lyrics" : lyricsvector , "SongInfo" : songinfovector, })
df.to_csv(".csv", index=False)

In [0]:
library(ggplot2)
library(tm)
library(RColorBrewer)
library(wordcloud)

#read in lyrics 
lyrics <- read.csv("textminingAllLyrics.csv", stringsAsFactors=FALSE, header=TRUE)

#split songinfo into artist and songname
lyrics$Artist <- sapply(strsplit(lyrics$SongInfo, ' - '), '[', 1)
lyrics$SongTitle <- sapply(strsplit(lyrics$SongInfo, ' - '), '[', 2)

#rename genres
lyrics$Genre[lyrics$Genre=="Country Music"] <- "Country"
lyrics$Genre[lyrics$Genre=="Hip Hop/Rap"] <- "Rap"

In [0]:
lyrics$SongTitle[duplicated(lyrics$SongTitle)]
lyrics <- subset(lyrics, !duplicated(lyrics$SongTitle))
table(lyrics$Genre)

In [0]:
lyrics$WordCount <- sapply(gregexpr("[[:alpha:]]+", lyrics$Lyrics), function(x) sum(x > 0))
summary(lyrics$WordCount)

#create bar graph of avg words by genre
genremean <- data.frame(genre=levels(as.factor(lyrics$Genre)),
                  meanwords=tapply(lyrics$WordCount, lyrics$Genre, mean))

ggplot(genremean, aes(x = factor(genre), y = meanwords)) + geom_bar(stat = "identity") +
  geom_text(aes(y=meanwords, ymax=800, label=round(meanwords,0)), 
                position= position_dodge(width=0.9), vjust=-.5, color="black") +
  scale_y_continuous("Average Wordcount",limits=c(0,800),breaks=seq(0, 800, 200)) + 
  scale_x_discrete("Genre")

In [0]:
corpus <- Corpus(VectorSource(lyrics$Lyrics))

corpus <- tm_map(corpus, tolower) #make words lower-case
corpus <- tm_map(corpus, PlainTextDocument) #remove formatting

corpus <- tm_map(corpus, removePunctuation) #remove punctuation
corpus <- tm_map(corpus, stripWhitespace) #remove any white space
corpus <- tm_map(corpus, removeWords, stopwords("english")) #remove stopwords
dtmcloud <- DocumentTermMatrix(corpus)
corpus <- tm_map(corpus, stemDocument, language="english") #stemming

# create document term matrix
dtm <- DocumentTermMatrix(corpus)
dtm #6762 unique terms

# Remove sparse terms
dtm <- removeSparseTerms(dtm, 0.98)
dtm #785 terms remain
freq <- sort(colSums(as.matrix(dtmcloud)), decreasing=TRUE)
dark2 <- brewer.pal(8, "Dark2")   
wordcloud(names(freq), freq, max.words=40, rot.per=0.2, colors=dark2)  

# plot top 20 words
wf <- data.frame(word=names(freq), freq=freq)   
ggplot(subset(wf, freq>450), aes(word, freq)) + 
  geom_bar(stat="identity") +  
  theme(axis.text.x=element_text(angle=45, hjust=1))  + 
  labs(list(title = "Frequency of Top 20 Words", x = "Word", y ="Frequency"))


In [0]:
library(caTools)
library(rpart)
library(rpart.plot)
library(randomForest)

# Create data frame
lyrics.tree = as.data.frame(as.matrix(dtm))
rownames(lyrics.tree) <- lyrics$SongInfo #rename each row as the associated song

In [0]:
#include genre and wordcount variables into data frame 
lyrics.tree$genre = lyrics$Genre
lyrics.tree$wordcount = lyrics$WordCount

# rename variables which are causing errors with randomForest
names(lyrics.tree)[names(lyrics.tree) == 'break'] <- 'break1'
names(lyrics.tree)[names(lyrics.tree) == 'next'] <- 'next1'
names(lyrics.tree)[names(lyrics.tree) == 'repeat'] <- 'repeat1'


In [0]:
set.seed(100)
spl = sample.split(lyrics.tree$genre, 0.70)
lyricsTrain = subset(lyrics.tree, spl == TRUE)
lyricsTest = subset(lyrics.tree, spl == FALSE)

In [0]:
# create decision tree model
lyricsCART = rpart(genre~., data=lyricsTrain, method="class")
prp(lyricsCART)
# predict on test set using CART
predCART = predict(lyricsCART, newdata=lyricsTest, type = "class")
table(lyricsTest$genre, predCART)

In [0]:
sum(diag(table(lyricsTest$genre, predCART)))/nrow(lyricsTest)

#Compute accuracy for christian
predCART.christian<-table(lyricsTest$genre=="Christian",predCART=="Christian")
predCART.christian[2,2]/sum(predCART.christian[2,])

#Compute accuracy for country
predCART.country<-table(lyricsTest$genre=="Country",predCART=="Country")
predCART.country[2,2]/sum(predCART.country[2,])

#Compute accuracy for rap
predCART.rap<-table(lyricsTest$genre=="Rap",predCART=="Rap")
predCART.rap[2,2]/sum(predCART.rap[2,])

#Compute accuracy for pop
predCART.pop<-table(lyricsTest$genre=="Pop",predCART=="Pop")
predCART.pop[2,2]/sum(predCART.pop[2,])

#Compute accuracy for R&B
predCART.rnb<-table(lyricsTest$genre=="R&B",predCART=="R&B")
predCART.rnb[2,2]/sum(predCART.rnb[2,])

#Compute accuracy for rock
predCART.rock<-table(lyricsTest$genre=="Rock",predCART=="Rock")
predCART.rock[2,2]/sum(predCART.rock[2,])

In [0]:
# create randomForest model
lyricsRF <- randomForest(as.factor(genre)~., data=lyricsTrain, importance=TRUE)
varImpPlot(lyricsRF)
# predict on test set using RF
predRF <- predict(lyricsRF, lyricsTest, type="response")
table(lyricsTest$genre, predRF)


In [0]:
sum(diag(table(lyricsTest$genre, predRF)))/nrow(lyricsTest)

#Compute accuracy for christian
predRF.christian<-table(lyricsTest$genre=="Christian",predRF=="Christian")
predRF.christian[2,2]/sum(predRF.christian[2,])

#Compute accuracy for country
predRF.country<-table(lyricsTest$genre=="Country",predRF=="Country")
predRF.country[2,2]/sum(predRF.country[2,])

#Compute accuracy for rap
predRF.rap<-table(lyricsTest$genre=="Rap",predRF=="Rap")
predRF.rap[2,2]/sum(predRF.rap[2,])

#Compute accuracy for pop
predRF.pop<-table(lyricsTest$genre=="Pop",predRF=="Pop")
predRF.pop[2,2]/sum(predRF.pop[2,])

#Compute accuracy for R&B
predRF.rnb<-table(lyricsTest$genre=="R&B",predRF=="R&B")
predRF.rnb[2,2]/sum(predRF.rnb[2,])

#Compute accuracy for rock
predRF.rock<-table(lyricsTest$genre=="Rock",predRF=="Rock")
predRF.rock[2,2]/sum(predRF.rock[2,])